In [10]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from selenium import webdriver #selenium 4.20.0
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time

 # version 4.23.1

In [21]:
csv_path = pd.read_csv(r"Sofa Score Data - Matches(6).csv")
csv_path = csv_path[csv_path['match_status'] == 'finished']
csv_path.reset_index(drop=True, inplace=True)
csv_path

urls = csv_path['url']
urls

0     https://www.sofascore.com/football/match/persi...
1     https://www.sofascore.com/football/match/dewa-...
2     https://www.sofascore.com/football/match/bhaya...
3     https://www.sofascore.com/football/match/rans-...
4     https://www.sofascore.com/football/match/persi...
5     https://www.sofascore.com/football/match/madur...
6     https://www.sofascore.com/football/match/perse...
7     https://www.sofascore.com/football/match/persi...
8     https://www.sofascore.com/football/match/bhaya...
9     https://www.sofascore.com/football/match/pss-s...
10    https://www.sofascore.com/football/match/persi...
11    https://www.sofascore.com/football/match/dewa-...
12    https://www.sofascore.com/football/match/madur...
13    https://www.sofascore.com/football/match/persi...
14    https://www.sofascore.com/football/match/rans-...
15    https://www.sofascore.com/football/match/borne...
16    https://www.sofascore.com/football/match/bhaya...
17    https://www.sofascore.com/football/match/p

In [31]:
# csv_path.to_csv('Sofa Score Data - Matches(3).csv', index=False)

In [22]:
# Define browser and action setup
PATH = "D:\\SOFTWARE DOWNLOAD\\chromedriver.exe"
service = ChromeService(executable_path=PATH)
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)
driver = webdriver.Chrome(service=service, options=options)

# # Action Steps
# driver.maximize_window()

In [23]:
all_stats = []

for url in urls:
    driver.set_page_load_timeout(90)

    try:
        driver.get(url)
    except:
        pass

    try:
        element = driver.find_element(By.XPATH, '//div[@class="Box hjanni"]')
    
        # Scroll
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
    
        # Click element
        element.click()

        # Wait until the pop up appears
        time.sleep(4)

    except Exception as e:
        print(f"Terjadi kesalahan pada URL {url}:", e)

    # extract requests from logs
    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

    desired_keywords = ['average-position']

    for log in logs:
        # Check if the log contains the 'method' key
        if 'method' in log:
            # Check if the log method is 'Network.responseReceived'
            if log["method"] == "Network.responseReceived":
                # Extract request ID and URL
                request_id = log["params"]["requestId"]
                response_url = log["params"]["response"]["url"]
                # Check if the URL contains any of the desired keywords
                if any(keyword in response_url for keyword in desired_keywords):
                    # Retrieve match_id from CSV based on URL
                    match_id_row = csv_path[csv_path['url'] == url]
                    match_id = match_id_row['match_id'].values[0] if not match_id_row.empty else 'unknown'

                    # Print the request ID, URL, and match id
                    print(f"Request ID: {request_id}, URL: {response_url}, Match ID: {match_id}")
                    
                    # Try to retrieve the response body using the request ID
                    try:
                        response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': request_id})
                        stats = response['body']
                        if stats:
                            print("Response body successfully retrieved:")
                            print(stats)
                            all_stats.append({"url": url, "match_id": match_id, "stats": stats})
                            break
                        else:
                            print("Response body is empty. Trying next request ID...")
                    except Exception as e:
                        print(f"Error occurred while retrieving response for request ID {request_id}: {e}")
                        continue

# driver.quit()

# Convert all_stats to DataFrame
all_stats_json = [{"url": item["url"], "match_id": item["match_id"], **json.loads(item["stats"])} for item in all_stats]
df = pd.json_normalize(all_stats_json)

# Print DataFrame
print("All Collected stats as DataFrame:")
print(df)

Terjadi kesalahan pada URL https://www.sofascore.com/football/match/persik-kediri-persib-bandung/OKAsqreb#id:11383004: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="Box hjanni"]"}
  (Session info: chrome=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D2F653+11395]
	(No symbol) [0x00D0B7A4]
	(No symbol) [0x00C51A43]
	(No symbol) [0x00C956C8]
	(No symbol) [0x00C958AB]
	(No symbol) [0x00CD26A2]
	(No symbol) [0x00CB71E4]
	(No symbol) [0x00CD03DC]
	(No symbol) [0x00CB6F36]
	(No symbol) [0x00C8A276]
	(No symbol) [0x00C8AECD]
	GetHandleVerifier [0x00E32403+1071667]
	GetHandleVerifier [0x00E3C9C4+1114100]
	GetHandleVerifier [0x00E354B2+1084130]
	GetHandleVerifier [0x00D5013B+145259]
	(No symbol) [0x00D11693]
	(No symbol) [0x00D0EFB8]
	(No symbol) [0x00D0F157]
	(No symbol) [0x00D0842E]
	BaseThreadI

In [25]:
df.to_csv("stats_result_till-11382966.csv")
df

,url,match_id,home,away,substitutions
0,https://www.sofascore.com/football/match/bhaya...,11382978,"[{'player': {'name': 'Wawan Febrianto', 'slug'...","[{'player': {'name': 'Awan Setho Raharjo', 'sl...","[{'playerIn': {'name': 'Luthfi Kamal', 'firstN..."
1,https://www.sofascore.com/football/match/psis-...,11382966,"[{'player': {'name': 'Fahreza Sudin', 'firstNa...","[{'player': {'name': 'Haykal Alhafiz', 'firstN...","[{'playerIn': {'name': 'Paulo Gali', 'firstNam..."


In [27]:
df1 = pd.read_csv("stats_result_till-12225691.csv")
df2 = pd.read_csv("stats_result_till-11543594.csv")
df3 = pd.read_csv("stats_result_till-11465542.csv")
df4 = pd.read_csv("stats_result_till-11465407.csv")
df5 = pd.read_csv("stats_result_till-11383063.csv")
df6 = pd.read_csv("stats_result_till-11383002.csv")
df7 = pd.read_csv("stats_result_till-11382966.csv")

In [38]:
# Drop "Unnamed" column if it exists
dfs = [df.drop(columns=[col for col in df.columns if "Unnamed" in col]) for df in [df1, df2, df3, df4, df5, df6, df7]]

combined_df = pd.concat(dfs, ignore_index=True)

combined_df.to_csv("AveragePosition.csv")
# Print the combined DataFrame
combined_df

,url,match_id,home,away,substitutions
0,https://www.sofascore.com/football/match/persi...,11543578,"[{'player': {'name': 'Ramiro Fergonzi', 'first...","[{'player': {'name': 'Tegar Infantri', 'firstN...","[{'playerIn': {'name': 'Jack Brown', 'firstNam..."
1,https://www.sofascore.com/football/match/psis-...,11543574,"[{'player': {'name': 'Andritany Ardhiyasa', 's...","[{'player': {'name': 'Tri Setiawan', 'firstNam...",[{'playerIn': {'name': 'Septian David Maulana'...
2,https://www.sofascore.com/football/match/pss-s...,11543571,"[{'player': {'name': 'Kim Jeffrey Kurniawan', ...","[{'player': {'name': 'Ferdiansyah Cecep', 'slu...","[{'playerIn': {'name': 'Edo Febriansyah', 'slu..."
3,https://www.sofascore.com/football/match/bali-...,12305963,"[{'player': {'name': 'Andhika Wijaya', 'slug':...","[{'player': {'name': 'Dedi Kusnandar', 'slug':...",[{'playerIn': {'name': 'Beckham Putra Nugraha'...
4,https://www.sofascore.com/football/match/borne...,12305961,"[{'player': {'name': 'Riyatno Abiyoso', 'slug'...","[{'player': {'name': 'Taufany Muslihuddin', 'f...",[{'playerIn': {'name': 'Bayu Gatra Sanggiawan'...
...,...,...,...,...,...
121,https://www.sofascore.com/football/match/psis-...,11382991,[{'player': {'name': 'Gian Zola Nasrulloh Nugr...,"[{'player': {'name': 'Costas Tatarakis', 'slug...","[{'playerIn': {'name': 'Bruno Moreira', 'slug'..."
122,https://www.sofascore.com/football/match/pss-s...,11382994,"[{'player': {'name': 'Yevhen Bokhashvili', 'fi...","[{'player': {'name': 'Carlos Fortes', 'slug': ...","[{'playerIn': {'name': 'Wawan Febrianto', 'slu..."
123,https://www.sofascore.com/football/match/borne...,11383002,"[{'player': {'name': 'Riyan Ardiansyah', 'firs...","[{'player': {'name': 'Jelle Goselink', 'slug':...","[{'playerIn': {'name': 'Riyan Ardiansyah', 'fi..."
124,https://www.sofascore.com/football/match/bhaya...,11382978,"[{'player': {'name': 'Wawan Febrianto', 'slug'...","[{'player': {'name': 'Awan Setho Raharjo', 'sl...","[{'playerIn': {'name': 'Luthfi Kamal', 'firstN..."
